In [ ]:
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.serialize import model_to_json, model_from_json
import pandas as pd
import json
from tqdm import tqdm
from datetime import date
from llaves_more_then_q95 import llaves_more_then_q95
import numpy as np
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
import logging

# Set the logging level for cmdstanpy to WARNING
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)


In [ ]:
df_grouped_by_week = pd.read_parquet('./datasets/almacenes_si_curated_by_week.parquet')

In [128]:
df_grouped_by_week[df_grouped_by_week['combination'] == '248AI6QJ6T10']

,combination,date_week,quantity,discount_for_event,campaigns_name,price_taxes_excluded,campaign
170356,248AI6QJ6T10,2018-01-08,1,0.0,no-discount,23521.009766,0
170357,248AI6QJ6T10,2018-01-15,5,0.0,no-discount,92403.359375,0
170358,248AI6QJ6T10,2018-01-29,2,0.0,no-discount,46201.679688,0
170359,248AI6QJ6T10,2018-02-05,5,0.0,no-discount,69722.687500,0
170360,248AI6QJ6T10,2018-02-19,5,0.0,TIJERETAZO I,115924.367188,1
...,...,...,...,...,...,...,...
170431,248AI6QJ6T10,2023-02-27,1,0.0,TIJERETAZO I,36546.000000,1
170432,248AI6QJ6T10,2023-03-06,2,0.0,TIJERETAZO I,34193.281250,1
170433,248AI6QJ6T10,2023-03-20,1,0.0,TIJERETAZO I,34193.281250,1
170434,248AI6QJ6T10,2023-03-27,2,0.0,no-discount,70739.500000,0


In [ ]:
df_grouped_by_week.head()

In [ ]:
archivo_insumo = df_grouped_by_week[['date_week','combination','campaign','discount_for_event']]
archivo_insumo = archivo_insumo[archivo_insumo['date_week'].between('2023-01-01','2023-12-31')]
archivo_insumo['familia'] = archivo_insumo['combination'].apply(lambda x: str(x)[:3])
archivo_insumo = archivo_insumo.groupby(['date_week','familia'], as_index=False)[['discount_for_event','campaign']].max()
archivo_insumo = archivo_insumo[['familia','date_week','discount_for_event','campaign']]

In [ ]:
archivo_insumo.head()

In [ ]:
archivo_campanias = df_grouped_by_week[['date_week','combination','campaign','discount_for_event']]
archivo_campanias['familia'] = archivo_campanias['combination'].apply(lambda x: str(x)[:3])
archivo_campanias = archivo_campanias.groupby(['date_week','familia'], as_index=False)[['discount_for_event','campaign']].max()
archivo_campanias = archivo_campanias[['familia','date_week','discount_for_event','campaign']]
archivo_campanias

In [ ]:
llaves_con_informacion_historica_suficiente_para_forecast = []
combinations_to_forecast = df_grouped_by_week['combination'].unique()
for combination in tqdm(combinations_to_forecast):
    df_combination = df_grouped_by_week[df_grouped_by_week['combination'] == combination]
    if len(df_combination[df_combination['date_week'] >= '2022-01-01']) >= 12: # si tiene por lo menos 12 semanas desde el 2022
        llaves_con_informacion_historica_suficiente_para_forecast.append(combination)
print(len(llaves_con_informacion_historica_suficiente_para_forecast))

In [140]:
df_grouped_by_week[df_grouped_by_week['combination'] == combination]

,combination,date_week,quantity,discount_for_event,campaigns_name,price_taxes_excluded,campaign
57230,240AG7PA5,2022-11-28,13,40.0,BLACK WEEK,109235.289062,1
57231,240AG7PA5,2023-08-07,1,0.0,no-discount,54617.648438,0
57232,240AG7PA5,2023-09-04,4,35.0,TIJERETAZO II,128557.140625,1
57233,240AG7PA5,2023-09-11,1,35.0,TIJERETAZO II,37810.921875,1
57234,240AG7PA5,2023-09-25,2,35.0,TIJERETAZO II,37810.921875,1
57235,240AG7PA5,2023-10-02,5,35.0,TIJERETAZO II,128557.140625,1
57236,240AG7PA5,2023-10-16,1,0.0,no-discount,45373.109375,0
57237,240AG7PA5,2023-10-23,1,0.0,no-discount,75621.851562,0
57238,240AG7PA5,2023-10-30,1,0.0,no-discount,54617.648438,0
57239,240AG7PA5,2023-11-13,1,0.0,no-discount,75621.851562,0


In [142]:
final_prediction_2023 = pd.DataFrame()
rmse_por_llave = pd.DataFrame()
test_model = True

for combination in tqdm(llaves_con_informacion_historica_suficiente_para_forecast):

    # Get the dataframe for the combination
    df_combination = df_grouped_by_week[df_grouped_by_week['combination'] == combination]
    df_combination = df_combination[['date_week','combination','quantity']]
    
    if combination in llaves_more_then_q95 and len(df_combination[df_combination['date_week'] >= '2022-01-01']) > 0: # si la llave esta en las llaves q95 y tiene data en el 2022, se entrena con data 2022, de lo contrario, se entra con todo lo disponible
        df_combination = df_combination[df_combination['date_week'] >= '2022-01-01']
    # df_combination_test = df_grouped_by_week_test[df_grouped_by_week_test['combination'] == combination][['date_week','quantity','campaign','discount_for_event']] # get the y_true values of the combination selected

    familia = str(combination[:3])
    df_discount_and_campaings = archivo_campanias[archivo_campanias['familia'] == familia]

    # df_discount_and_campaings = df_grouped_by_week[df_grouped_by_week['date_week'] >= '2024-01-01']
    # df_discount_and_campaings = df_discount_and_campaings[['combination','date_week','discount_for_event','campaign']]
    # Complete the missing weeks with 0
    first_date = df_combination['date_week'].min() # Get the first date in the dataframe
    last_date = df_combination['date_week'].max() # Get the last date in the dataframe
    
    if test_model == True and first_date.year <= 2022:
        # Create a dataframe with all the weeks between the year of the first date and the year of the last date
        df_dates = pd.DataFrame({'date_week': pd.date_range(start=f'{first_date.year}-01-08', end=f'2022-12-31', freq='W-MON')})    

        df_combination = df_dates.merge(df_combination, on='date_week', how='left') # Merge the dataframes
        df_combination['quantity'] = df_combination['quantity'].fillna(0) # Replace NaN values with 0
        df_combination = df_combination[~(df_combination['combination'].isnull())]
        df_combination = df_combination.merge(df_discount_and_campaings, on='date_week', how='left') # Merge the dataframes

        # Create the prophet dataframe
        prophet_dataframe = pd.DataFrame()
        prophet_dataframe['ds'] = df_combination['date_week']
        prophet_dataframe['y'] = df_combination['quantity']
        prophet_dataframe['campaign'] = df_combination['campaign']
        prophet_dataframe['discount_for_event'] = df_combination['discount_for_event']
        prophet_dataframe['y'] = prophet_dataframe['y'].astype(int)

        n_unique_years = prophet_dataframe['ds'].dt.year.nunique() # Count the number of different years in the dataframe

        # Create the model
        if n_unique_years == 1:
            model = Prophet(weekly_seasonality = 13)
            model.add_regressor('campaign')
            model.add_regressor('discount_for_event')

        else:
            model = Prophet(weekly_seasonality = 52)
            model.add_regressor('campaign')
            model.add_regressor('discount_for_event')
            
            
        try:
            model.fit(prophet_dataframe)
            with open(f'./serialized_models/{str(combination)}.json', 'w') as file:
                file.write(model_to_json(model))  # Save model
            
            
            if test_model:
                # Calculate how many weeks are missing from the last date in the dataframe to the last week of 2023
                weeks = (date(2023, 12, 31) - prophet_dataframe['ds'].max().date()).days // 7

                # Create a dataframe with the dates from the last date in the dataframe to the last week of 2023
                df_grouped_by_week_test = df_grouped_by_week[df_grouped_by_week['date_week'].between('2023-01-01','2023-12-31')]
                df_combination_test = df_grouped_by_week_test[df_grouped_by_week_test['combination'] == combination][['date_week','quantity']] # get the y_true values of the combination selected
                df_combination_test = df_combination_test.merge(df_discount_and_campaings, on='date_week', how='left') # Merge the dataframes

                future_2023 = model.make_future_dataframe(periods=weeks, freq='W-MON')
                future_2023 = future_2023[future_2023['ds'] >= '2023-01-01']
                future_2023 = future_2023.merge(df_combination_test, right_on='date_week',left_on= 'ds', how = 'left')
                del future_2023['date_week']
                future_2023['familia'] = familia
                future_2023['campaign'].fillna(0,inplace=True)
                future_2023['quantity'].fillna(0,inplace=True)
                future_2023['discount_for_event'].fillna(0,inplace=True)
                
                
                forecast = model.predict(future_2023) # Make the predictions
                forecast_2023 = forecast[['ds', 'yhat']] # Get the predictions for 2023
                forecast_2023.columns = ['date', 'demand_yhat'] # Rename columns
                forecast_2023['llave'] = combination # Add the combination column
                # Add to the final predictions dataframe
                final_prediction_2023 = pd.concat([final_prediction_2023, forecast_2023])
                final_prediction_2023['demand_yhat'] = final_prediction_2023['demand_yhat'].apply(lambda x: 0 if x < 0 else x)
                final_prediction_2023['demand_yhat'] = final_prediction_2023['demand_yhat'].apply(lambda x: np.ceil(x))

                X_test = df_grouped_by_week[df_grouped_by_week['combination'] == combination][['date_week','quantity']]
                X_test.columns = ['date','y_true']
                df_ytrue_yhat = final_prediction_2023.merge(X_test,on= 'date', how = 'left')
                df_ytrue_yhat['y_true'].fillna(0, inplace=True)

                RMSE = np.sqrt(mean_squared_error(df_ytrue_yhat['y_true'], df_ytrue_yhat['demand_yhat']))
                dict_temp = {
                    'llave' : combination,
                    'rsme' : RMSE
                }
                df_rmse = pd.DataFrame([dict_temp])
                rmse_por_llave = pd.concat([rmse_por_llave,df_rmse])
        except Exception as e:
            print(f'problems with this key: {combination}\n{e}')



        
    else:
        print(combination)

  0%|          | 0/3337 [00:00<?, ?it/s]

  2%|▏         | 67/3337 [00:21<16:07,  3.38it/s]  

219301AC1


  2%|▏         | 71/3337 [00:22<13:30,  4.03it/s]

219301AU4


...

 18%|█▊        | 611/3337 [03:41<28:33,  1.59it/s]

248AF5QC5T16
248AF5QC5TM
248AF5QC5TS
248AF5QC5TXL
248AF5QD1T16


 21%|██        | 688/3337 [04:14<04:05, 10.80it/s]

248AF5QT1T08
248AF5QT1T10
248AF5QT1T12
248AF5QT1T14
248AF5QT1T16
248AF5QT1TL
248AF5QT1TM
248AF5QT1TS
248AF5QT3T14
248AF5QT3T16
248AF5QT3TM
248AF5QT3TS


 47%|████▋     | 1585/3337 [14:18<18:54,  1.54it/s]

In [ ]:
df_combination

In [ ]:
rmse_por_llave

In [ ]:

    # # -----------------------------------------------------------------------------------
    # # Do the same to generate de prophet_df, but with test data(2023) to compare results
    # # -----------------------------------------------------------------------------------
    # # Complete the missing weeks with 0
    # first_date_test = df_combination_test['date_week'].min() # Get the first date in the dataframe
    # last_date_test = df_combination_test['date_week'].max() # Get the last date in the dataframe
    # # Create a dataframe with all the weeks between the year of the first date and the year of the last date
    # df_dates_test = pd.DataFrame({'date_week': pd.date_range(start=f'{first_date_test.year}-01-08', end=f'{last_date_test.year}-12-31', freq='W-MON')})    
    # df_combination_test = df_dates_test.merge(df_combination_test, on='date_week', how='left') # Merge the dataframes
    # df_combination_test['quantity'] = df_combination_test['quantity'].fillna(0) # Replace NaN values with 0
    # df_combination_test = df_combination_test[df_combination_test['date_week'] < '2024-01-01'] # Drop registers from 2024
    
    # # Create the prophet TEST dataframe
    # dataframe_test = pd.DataFrame()
    # dataframe_test['ds'] = df_combination_test['date_week']
    # dataframe_test['y_true'] = df_combination_test['quantity']

    # # Merge the forecast_2024 with the y_true values
    # prediction = forecast[['ds', 'yhat']].merge(dataframe_test, on='ds', how='left')
    # prediction = prediction[prediction['ds'] >= '2023-01-01']
    # prediction['y_true'] = prediction['y_true'].fillna(0) # Fill NaN values with 0
    # prediction['y_true'] = prediction['y_true'].astype(int)
    # prediction['yhat'] = prediction['yhat'].apply(lambda x: 0 if x < 0 else x) # Replace negative values with 0
    
    
    # # Calculate the rsme for test_set
    # df_rsme = prediction.copy()
    # rsme = np.sqrt(mean_squared_error(df_rsme['y_true'], df_rsme['yhat']))
    
    # # generate new column error with the abs(error)
    # prediction['error'] = prediction['y_true'] - prediction['yhat']
    # prediction['error'] = prediction['error'].apply(lambda x: abs(x))
    # prediction['llave'] = combination
    # final_prediction_2024_with_test_data = pd.concat([final_prediction_2024_with_test_data,prediction])
    # rmse_por_llave = {
    #     'llave': combination,
    #     'rmse' : rsme
    # }
    # rmse_por_llave_resultados.append(rmse_por_llave)
    with open(f'.././serialized_models/{str(combination)}.json', 'w') as file:
        file.write(model_to_json(model))  # Save model



In [ ]:
final_prediction_2024 = pd.DataFrame()
final_prediction_2024_with_test_data = pd.DataFrame()
rmse_por_llave_resultados = []
# Get the list of combinations
for combination in tqdm(llaves_con_informacion_historica_suficiente_para_forecast):
    # Get the dataframe for the combination
    df_combination = df_grouped_by_week[df_grouped_by_week['combination'] == combination]
    if combination in llaves_more_then_q95 and len(df_combination[df_combination['date_week'] >= '2022-01-01']) > 0: # si la llave esta en las llaves q95 y tiene data en el 2022, se entrena con data 2022, de lo contrario, se entra con todo lo disponible
        df_combination = df_combination[df_combination['date_week'] >= '2022-01-01']
    # df_combination_test = df_grouped_by_week_test[df_grouped_by_week_test['combination'] == combination][['date_week','quantity','campaign','discount_for_event']] # get the y_true values of the combination selected
    df_discount_and_campaings = df_grouped_by_week[df_grouped_by_week['date_week'] >= '2024-01-01']
    df_discount_and_campaings = df_discount_and_campaings[['combination','date_week','discount_for_event','campaign']]
    # Complete the missing weeks with 0
    first_date = df_combination['date_week'].min() # Get the first date in the dataframe
    last_date = df_combination['date_week'].max() # Get the last date in the dataframe
    # Create a dataframe with all the weeks between the year of the first date and the year of the last date
    df_dates = pd.DataFrame({'date_week': pd.date_range(start=f'{first_date.year}-01-08', end=f'{last_date.year}-12-31', freq='W-MON')})    
    
    df_combination = df_dates.merge(df_combination, on='date_week', how='left') # Merge the dataframes
    df_combination['quantity'] = df_combination['quantity'].fillna(0) # Replace NaN values with 0
    df_combination['campaign'] = df_combination['campaign'].fillna(0) # Replace NaN values with 0
    df_combination['discount_for_event'] = df_combination['discount_for_event'].fillna(0) # Replace NaN values with 0

    df_combination = df_combination[df_combination['date_week'] < '2023-12-31'] # Drop registers from 2024

    # Create the prophet dataframe
    prophet_dataframe = pd.DataFrame()
    prophet_dataframe['ds'] = df_combination['date_week']
    prophet_dataframe['y'] = df_combination['quantity']
    prophet_dataframe['campaign'] = df_combination['campaign']
    prophet_dataframe['discount_for_event'] = df_combination['discount_for_event']
    prophet_dataframe['y'] = prophet_dataframe['y'].astype(int)

    n_unique_years = prophet_dataframe['ds'].dt.year.nunique() # Count the number of different years in the dataframe

    # Create the model
    if n_unique_years == 1:
        model = Prophet(weekly_seasonality = 13)
        model.add_regressor('campaign')
        model.add_regressor('discount_for_event')

    else:
        model = Prophet(weekly_seasonality = 52)
        model.add_regressor('campaign')
        model.add_regressor('discount_for_event')
        
        
    model.fit(prophet_dataframe)

    # Calculate how many weeks are missing from the last date in the dataframe to the last week of 2023
    weeks = (date(2024, 12, 31) - prophet_dataframe['ds'].max().date()).days // 7
    # Create a dataframe with the dates from the last date in the dataframe to the last week of 2023
    future_2024 = model.make_future_dataframe(periods=weeks, freq='W-MON')
    future_2024 = future_2024[future_2024['ds'] >= '2024-01-01']
    future_2024 = future_2024.merge(df_combination_test, right_on='date_week',left_on= 'ds', how = 'left')
    del future_2024['date_week']
    future_2024[['quantity','campaign','discount_for_event']] = future_2024[['quantity','campaign','discount_for_event']].fillna(0)



    forecast = model.predict(future_2024) # Make the predictions
    forecast_2024 = forecast[['ds', 'yhat']] # Get the predictions for 2023
    forecast_2024.columns = ['date', 'demand_yhat'] # Rename columns
    forecast_2024['llave'] = combination # Add the combination column
    # Add to the final predictions dataframe
    final_prediction_2024 = pd.concat([final_prediction_2024, forecast_2024])
    final_prediction_2024['demand_yhat'] = final_prediction_2024['demand_yhat'].apply(lambda x: 0 if x < 0 else x)
    # # -----------------------------------------------------------------------------------
    # # Do the same to generate de prophet_df, but with test data(2023) to compare results
    # # -----------------------------------------------------------------------------------
    # # Complete the missing weeks with 0
    # first_date_test = df_combination_test['date_week'].min() # Get the first date in the dataframe
    # last_date_test = df_combination_test['date_week'].max() # Get the last date in the dataframe
    # # Create a dataframe with all the weeks between the year of the first date and the year of the last date
    # df_dates_test = pd.DataFrame({'date_week': pd.date_range(start=f'{first_date_test.year}-01-08', end=f'{last_date_test.year}-12-31', freq='W-MON')})    
    # df_combination_test = df_dates_test.merge(df_combination_test, on='date_week', how='left') # Merge the dataframes
    # df_combination_test['quantity'] = df_combination_test['quantity'].fillna(0) # Replace NaN values with 0
    # df_combination_test = df_combination_test[df_combination_test['date_week'] < '2024-01-01'] # Drop registers from 2024
    
    # # Create the prophet TEST dataframe
    # dataframe_test = pd.DataFrame()
    # dataframe_test['ds'] = df_combination_test['date_week']
    # dataframe_test['y_true'] = df_combination_test['quantity']

    # # Merge the forecast_2024 with the y_true values
    # prediction = forecast[['ds', 'yhat']].merge(dataframe_test, on='ds', how='left')
    # prediction = prediction[prediction['ds'] >= '2023-01-01']
    # prediction['y_true'] = prediction['y_true'].fillna(0) # Fill NaN values with 0
    # prediction['y_true'] = prediction['y_true'].astype(int)
    # prediction['yhat'] = prediction['yhat'].apply(lambda x: 0 if x < 0 else x) # Replace negative values with 0
    
    
    # # Calculate the rsme for test_set
    # df_rsme = prediction.copy()
    # rsme = np.sqrt(mean_squared_error(df_rsme['y_true'], df_rsme['yhat']))
    
    # # generate new column error with the abs(error)
    # prediction['error'] = prediction['y_true'] - prediction['yhat']
    # prediction['error'] = prediction['error'].apply(lambda x: abs(x))
    # prediction['llave'] = combination
    # final_prediction_2024_with_test_data = pd.concat([final_prediction_2024_with_test_data,prediction])
    # rmse_por_llave = {
    #     'llave': combination,
    #     'rmse' : rsme
    # }
    # rmse_por_llave_resultados.append(rmse_por_llave)
    with open(f'.././serialized_models/{str(combination)}.json', 'w') as file:
        file.write(model_to_json(model))  # Save model

